In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#hide
! pip install nbdev

In [ ]:
#default_exp utils

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import os
import re
import imghdr
import tensorflow as tf
import requests
from PIL import Image
from tqdm import notebook
from tensorflow.keras.callbacks import Callback
from tensorflow.train import CheckpointManager
from typing import *
from pathlib import Path
from shutil import copyfile

In [ ]:
#export
def copy_data(source:Path, dest:Path, copy_rate:float):
    ''' Copy data from source path to dest (destination) path
        Args:
            soucre: path of the data source
            dest: path where to copy the data
            copy_rate: percentage of how many data to copy from the source
    '''
    fnames = os.listdir(source)
    data_size = len(fnames)
    train_data_size = int(data_size * copy_rate)
    for idx, fname in enumerate(fnames):
        src = source / fname
        if idx + 1 > train_data_size:
            break
        if os.path.getsize(src) == 0 or imghdr.what(src) != 'jpeg':
            print("Ignoring {}, because it's a corrupted file".format(fname))
            continue
        copyfile(src, dest / fname)

In [ ]:
# #hide
# def how_many_in_each_class(root_dir ,classes):
#     output = {}
#     for class_name in classes:
#         output[class_name.upper()] = len(os.listdir(root_dir / class_name))
#     return output

In [ ]:
#export
def how_many_in_each_class(root_dir:Path,classes:List[str]):
    for class_name in classes:
        print('class: {} has: {} images'.format(class_name.upper(),len(os.listdir(root_dir / class_name))))    

In [ ]:
#export
def freeze_unfreeze_layers(model:tf.keras.Model, layers:List[str] = [], freeze_mode:bool = True):
    '''freeze unfreeze layers for a given model
        Args:
            model: The model to freeze unfreeze its layers
            layers: a list of layers to be frozen unfrozen
                    empty list means the operation will be 
                    applied to all layers of the given model
            freeze_mode: True to freeze the layers,
                         False to unfreeze the layers
    '''
    trainable = not(freeze_mode)
    if len(layers) == 0:
        for layer in model.layers:
            layer.trainable = trainable
        return
    for layer in layers:
        model.get_layer(layer).trainable = trainable

In [ ]:
#export
def list_layers(model:tf.keras.Model):
    ''' List all layers of a given model '''
    for layer in model.layers:
        print('name: {}, trainable: {}'.format(layer.name, layer.trainable))

In [ ]:
#export
def evaluate(model:tf.keras.Model, test_ds:tf.data.Dataset, metric:tf.metrics.Metric, num_batches:int = None):
    prog_bar = tf.keras.utils.Progbar(target=num_batches)
    for idx, batch in enumerate(test_ds):
        metric(batch[1], model(batch[0], training=False))
        prog_bar.update(idx)
    print()
    print("{}: {:.2f}".format(metric.name, metric.result()))

In [ ]:
#export
class CheckPointManagerCallback(Callback):
    ''' Keras Callback for the tf.train.CheckPointManager
        All arguments are same as in the tf.train.CheckPointManager
        except the `after_num_epoch`
        Args: 
            after_num_epoch: number of epochs between each check point save 
    '''
    def __init__(self, 
                checkpoint,
                directory,
                max_to_keep,
                after_num_epoch=1,
                keep_checkpoint_every_n_hours=None,
                checkpoint_name="ckpt",
                step_counter=None,
                checkpoint_interval=None,
                init_fn=None):
        super().__init__()
        self.manager = (CheckpointManager(checkpoint,  
                                          directory, 
                                          max_to_keep, 
                                          keep_checkpoint_every_n_hours, 
                                          checkpoint_name, 
                                          step_counter, 
                                          checkpoint_interval, 
                                          init_fn))
        self.epoch_counter = 0
        self.after_num_epoch = after_num_epoch
    
    def on_epoch_end(self,batch, logs={}):
        self.epoch_counter += 1
        if self.epoch_counter % self.after_num_epoch == 0:
            self.manager.save()

In [ ]:
#hide
import time

def timeit(dataset, steps=100):
    start_time = time.time()

    iterator = iter(dataset)

    for step in range(steps):
        next_batch = next(iterator)
        if step % 10 == 0:
            print('.', end='')
    print()

    end_time = time.time()
    duration = (end_time - start_time)
    
    print('{} batches: {} s'.format(steps, (duration)))
    print('{:.4f} Images/s'.format(batch_size * steps / duration))

In [ ]:
#export
def _validate_url(url):
    regex = re.compile(
            r'^(?:http|ftp)s?://' # http:// or https://
            r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|' #domain...
            r'localhost|' #localhost...
            r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ...or ip
            r'(?::\d+)?' # optional port
            r'(?:/?|[/?]\S+)$', re.IGNORECASE)
    return re.match(regex, url) is not None

In [ ]:
#export
def download_images(dest, file, counter = 0):
    urls = open(file).read().strip().split("\n")
    for idx, url in enumerate(urls):
        if not(_validate_url(url)): continue
        resp = requests.get(url, allow_redirects=False)
        suffix = '.jpg'
        img_name = str(idx + counter)
        img_full_name = str(dest/img_name) + suffix
        open(img_full_name,'wb').write(resp.content)

In [ ]:
#export
def verify_images(dest:Path, delete:bool=False, n_channels:int=3):
    fnames = os.listdir(str(dest))
    for fname in notebook.tqdm(fnames ,total=len(fnames)):
        if not(verify_image(dest/fname, n_channels)):
            if delete:
                os.remove(dest/fname)
                print('{} => corrupted image, so it was deleted'.format(dest/fname))
                continue
            print('{} => corrupted image, pass `delete=True` to delete corrupted images'.format(dest/fname))

In [ ]:
#export
def verify_image(img_path, n_channels):
    try:
        img = Image.open(img_path)
        img.draft(img.mode,(28,28))
        img.load()
        return imghdr.what(img_path) != None and  img.layers == n_channels
    except: 
        return False

In [ ]:
! nbdev_build_lib

Converted 00_utils.ipynb.
Converted Custom_Training_with_Custom_Loss_MNIST_FASHION.ipynb.
Converted Flowers_Classifier.ipynb.
Converted Improved_Data_Pipelines_for_Small_Datasets_Tensorflow_2.x.ipynb.
Converted index.ipynb.


In [ ]:
! ssh-keygen -t rsa -b 2048

Generating public/private rsa key pair.
Enter file in which to save the key (/root/.ssh/id_rsa): 
Created directory '/root/.ssh'.
Enter passphrase (empty for no passphrase): 159753
Enter same passphrase again: 159753
Your identification has been saved in /root/.ssh/id_rsa.
Your public key has been saved in /root/.ssh/id_rsa.pub.
The key fingerprint is:
SHA256:uB5I2ND1myPqQNVwhF8773frwu80OtnGSrxhepzT5T0 root@83f1cf00cf8c
The key's randomart image is:
+---[RSA 2048]----+
|    .o+          |
|   ..= ..        |
|  . o.....       |
|   =  ..oo       |
|  o o o So       |
| . . o o ...    .|
|  . o o  . o=*oo.|
|   o . .  .+@=*Eo|
|    . .   .o+@=..|
+----[SHA256]-----+


In [ ]:
! cd /root/.ssh && cat id_rsa.pub

ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDGTE+YTqLtr5wQvTVqR7K3IXV0MTBsfxIeT03ocBLXzakiOVL9aqlhYxmpjVhKyKZWV5U1NDPo0901ycvg9nYogDsfJ9+3wkJawR03O0yHbNRX9yO/9sJnUaOT9YtKN/QpFAC7rG+3+248tqPJNbrGjutkHeRWCSeaBRJcl6i4KjjlgTPiri0q/YepbgfYCyCmMKF6uOe97DNv3EMsUc3tCc1vgWPHYHF31vgfteG1kQh4VydJLLdmLgalk1V1Vpab5q3g0aD7CLMCRR4Rad3JSvNuGwuSNqWNldXAz/JxG4Tc7py1hNHNOSuwGKzy9Hp7f/Cx52V1aIkgX9TeZh3D root@83f1cf00cf8c


In [ ]:
! nbdev_install_git_hooks

Executing: git config --local include.path ../.gitconfig
Success: hooks are installed and repo's .gitconfig is now trusted


In [ ]:
! git 